<a href="https://colab.research.google.com/github/BrookemWalters/pizza/blob/main/ProjectFour_mpi_BrookeWalters_04162024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.5-cp310-cp310-linux_x86_64.whl size=2746514 sha256=473429e90ba0ec6e6f46931410e5068d9693c98ef28076220d9474221b6a9089
  Stored in directory: /root/.cache/pip/wheels/18/2b/7f/c852523089e9182b45fca50ff56f49a51eeb6284fd25a66713
Successfully built mpi4py


In [ ]:
#brooke walters
#cis 677
# project four
# due 4/16/2024

# write the source file to a python file
%%writefile mm_mpip4.py
# import libraries
from mpi4py import MPI
import time
import sys
import numpy as np

# generate matrices
def random_matrices(n):
  a = np.random.randint(1, 200, size=(n, n))
  b = np.random.randint(1, 200, size=(n, n))
  c = np.zeros((n,n))

  return a,b,c

if __name__ == "__main__":
  # initialize mpi
    comm = MPI.COMM_WORLD # initialize MPI communicator
    rank = comm.Get_rank() #proccess's unique ID
    size = comm.Get_size()    #communicator's scale

    if rank == 0:
      #n will be determined by the user
      n = (int)(sys.argv[1])

      # generate matrices
      a,b,c = random_matrices(n)

      # divide the calculations for the resulting matrices by rows. Every worker computes a separate set of rows
      # determine the number of rows
      portion_each_node = n // size
      remainder_portion = n % size

    else:
      n = None

    # Broadcast the size of matrix b, the matric to be scattered
    # in the multiset to all participating nodes
    a = comm.bcast(a,root=0)

    # Define row indices for node processes
    # adapted from Running MPI Jobs with Conda+Python, https://rchau.uah.edu/gettingStarted/examples/conda-matrix-mult
    start = rank * portion_each_node + min(rank, remainder_portion)
    end = start + portion_each_node + (1 if rank < remainder_portion else 0)

    # scatter matrix b to processes
    if rank == 0:
      b_worker = [b[start: end] for start, end in zip(
      range(0, n, portion_each_node),
      range(portion_each_node, n + portion_each_node, portion_each_node))]

    else:
      b_worker = None

    b_row = comm.scatter(b_worker, root=0)

    # Each process computes its part of the result
    start_time = time.time()
    separate_set = np.matmul(a,b_row,out=c)
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Gather the partial results back to the root process
    result = comm.gather(separate_set, root=0)

    print(f"Process {rank} of {size}:  When n = {n}:"
    f"\n when the size and shape of the matrix c is {c.size}, {c.shape},"
    f" the computation time is: {elapsed_time} seconds\n")





Overwriting mm_mpip4.py


In [ ]:
# 300,400,500,600 and 700.

!python mm_mpip4.py 300
!python mm_mpip4.py 400
!python mm_mpip4.py 500
!python mm_mpip4.py 600
!python mm_mpip4.py 700

Process 0 of 1:  When n = 300:
 when the size and shape of the matrix c is 90000, (300, 300), the computation time is: 0.04860949516296387 seconds

Process 0 of 1:  When n = 400:
 when the size and shape of the matrix c is 160000, (400, 400), the computation time is: 0.13546442985534668 seconds

Process 0 of 1:  When n = 500:
 when the size and shape of the matrix c is 250000, (500, 500), the computation time is: 0.19206786155700684 seconds

Process 0 of 1:  When n = 600:
 when the size and shape of the matrix c is 360000, (600, 600), the computation time is: 0.3263084888458252 seconds

Process 0 of 1:  When n = 700:
 when the size and shape of the matrix c is 490000, (700, 700), the computation time is: 0.5328958034515381 seconds

